# 残差神经网络综述

AlexNet的提出开启了卷积神经网络应用的先河，随后的GoogleNet、VGG等网络使用了更小的卷积核并加大了深度，证明了卷积神经网络在处理图像问题方面具有更加好的性能；

但是随着层数的不断加深，卷积神经网络也暴露出来许多问题：

- 理论上讲，层数越多、模型越复杂，其性能就应该越好；但是实验证明随着层数的不断加深，性能反而有所下降。
- 深度卷积网络往往存在着梯度消失/梯度爆炸的问题；由于梯度反向传播过程中，如果梯度都大于1，则每一层大于1的梯度会不断相乘，使梯度呈指数型增长；同理如果梯度都小于1，梯度则会逐渐趋于零；使得深度卷积网络难以训练。
- 训练深层网络时会出现退化：随着网络深度的增加，准确率达到饱和，然后迅速退化。

而ResNet提出的残差结构，则一定程度上缓解了模型退化和梯度消失问题：

![](图片/残差神经网络图1.png)

作者提出，在一个结构单元中，如果我们想要学习的映射本来是y=H(x)，那么跟学习y=F(x)+x这个映射是等效的；这样就将本来回归的目标函数H(x)转化为F(x)+x，即F(x) = H(x) - x，称之为残差。

于是，ResNet相当于将学习目标改变了，不再是学习一个完整的输出，而是目标值H(x)和x的差值，即去掉映射前后相同的主体部分，从而突出微小的变化，也能够将不同的特征层融合。而且y=F(x)+x在反向传播求导时，x项的导数恒为1这样也解决了梯度消失问题。


# 残差神经网络详解

## 论文地址

[《Deep Residual Learning for Image Recognition》](https://arxiv.org/pdf/1512.03385v1.pdf)

## 核心思想
将本来回归的目标函数H(x)转化为F(x)+x，即F(x) = H(x) - x，称之为残差

## 残差单元
ResNet的基本的残差单元如图所示：

![](图片/残差神经网络图1.png)

基本结构如图，假设每个单元输入的特征层为x，经过两个卷积层获得输出y，将x与y求和即得到了这个单元的输出；

在训练时，我们将该单元目标映射（即要趋近的最优解）假设为F(x) + x，而输出为y+x，那么训练的目标就变成了使y趋近于F(x)。即去掉映射前后相同的主体部分x，从而突出微小的变化（残差）。

用数学表达式表示为：

$$ y = F(x,\{W_i\}) + W_sx $$

其中：

- x是残差单元的输入；
- y是残差单元的输出；
- F(x)是目标映射；
- {Wi}是残差单元中的卷积层；
- Ws是一个1x1卷积核大小的卷积，作用是给x降维或升维，从而与输出y大小一致（因为需要求和）；

## 改进单元
同时也可以进一步拓展残差结构：

![](图片/残差神经网络图2.png)

原论文中则以VGG为例：

![](图片/残差神经网络图3.png)

从VGG的19层，拓展到了34层。

可见使用了残差单元可以大大加深卷积神经网络的深度，而且不会影响性能和训练速度.


# 源代码

In [1]:
# 先导入一堆的库
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, Sequential


In [3]:
# 图形中，我们可以将这个神经网络分成不同的块，就是如下的


class BasicBlock(layers.Layer):
    """这个继承自Layer的，是单独的一个Layer"""
    
    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__() #  调用父类的
        
        # 这个类有两个卷积
        self.conv1 = layers.Conv2D(filter_num, (3, 3), strides=stride, padding='same')
        self.bn1 = layers.BatchNormalization()  # 批量规范化 该层在每个batch上将前一层的激活值重新规范化，即使得其输出数据的均值接近0，其标准差接近1
        self.relu = layers.Activation('relu')   # 激活函数
 
        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()
 
        # 判断移动步长的
        if stride != 1:
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=stride))
            #self.downsample = layers.Conv2D(filter_num, (1,1), strides = stride)
        else:
            self.downsample = lambda x:x
    def call(self, inputs, training=None):
 
        # [b, h, w, c]
        # 如下是构造残差神经网络，
        out = self.conv1(inputs) # 先卷积
        out = self.bn1(out)      # 然后批量规范化
        out = self.relu(out)     # 然后激活函数
 
        out = self.conv2(out)    # 再卷积
        out = self.bn2(out)      # 再批量规范化
 
        identity = self.downsample(inputs)  # 将input转成跟out相同的维度
 
        output = layers.add([out, identity])  # 相加
        output = tf.nn.relu(output)           # 激活函数
 
        return output


# 然后构造总体的残差神经网络

class ResNet(keras.Model):
    """残差神经网络类，是一个模型"""
 
    def __init__(self, layer_dims, num_classes=100): # [2, 2, 2, 2]
        super(ResNet, self).__init__()
        # 做一个
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)),
                                layers.BatchNormalization(),
                                layers.Activation('relu'),
                                layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same')
                                ])
 
        self.layer1 = self.build_resblock(64,  layer_dims[0])
        self.layer2 = self.build_resblock(128, layer_dims[1], stride=2)
        self.layer3 = self.build_resblock(256, layer_dims[2], stride=2)
        self.layer4 = self.build_resblock(512, layer_dims[3], stride=2)
 
        # output: [b, 512, h, w],
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes)
 
    def build_resblock(self, filter_num, blocks, stride=1):
 
        res_blocks = Sequential()
        # may down sample
        res_blocks.add(BasicBlock(filter_num, stride))
        # 每一块有多少个
        for _ in range(1, blocks):
            res_blocks.add(BasicBlock(filter_num, stride=1))
 
        return res_blocks
 
    def call(self, inputs, training=None):
 
        x = self.stem(inputs)
 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
 
        # [b, c]
        x = self.avgpool(x)
        # [b, 100]
        x = self.fc(x)
 
        return x
def resnet18():
    return ResNet([2, 2, 2, 2])

# 引用

>[残差神经网络ResNet系列网络结构详解：从ResNet到DenseNet](https://blog.csdn.net/weixin_44936889/article/details/103774753)